In [11]:
from keras.models import load_model
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
import keras
from tflearn.layers import conv_1d, embedding, max_pool_1d, fully_connected, flatten, regression, input_data, dropout
import tflearn
import tflearn
import time

In [14]:
variable = pd.read_csv('train.tsv', sep='\t')

In [15]:
variable.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [16]:
variable = variable.drop(['PhraseId', 'SentenceId'], axis=1)

In [17]:
variable.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2


In [18]:
trainx = variable['Phrase'].values
trainy = variable['Sentiment'].values

In [19]:
one_hot_labels = keras.utils.to_categorical(trainy, num_classes=5)
num_feature_map = 256
nodes = 1024
epochs = 10
split = 2072

vocab = set()
for review in trainx[:1000]:
    for word in review.split(' '):
        vocab.add(word)
vocab_size = len(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

def encode(text):
    vector = []
    for word in text.split(' '):
        vector.append(word2index[word])
    return vector

reviews = []
for review in trainx[:1000]:
    reviews.append(encode(review))
reviews = pad_sequences(reviews, maxlen=100, value=0.)


In [9]:
model = input_data(shape=[None, 100])
model = embedding(model, input_dim=vocab_size, output_dim=128)
conv1 = conv_1d(model, 50, filter_size=3, padding='same', activation='relu', regularizer='L2')
conv2 = conv_1d(conv1, 50, filter_size=4, padding='same', activation='relu', regularizer='L2')
pool = max_pool_1d(conv2, kernel_size=3)
pool = dropout(pool, 0.5)
pool = flatten(pool)
pool = fully_connected(pool, 5, activation='softmax')
reg = regression(pool, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy')
mod = tflearn.DNN(reg, tensorboard_dir='./tflearn-grapgh')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
start = time.time()
mod.fit(reviews,one_hot_labels, n_epoch=5, shuffle=True, show_metric=True, batch_size=64)
end = time.time()
print('Time taken - ', end-start)

Training Step: 79  | total loss: 0.75583 | time: 1.330s
| Adam | epoch: 005 | loss: 0.75583 - acc: 0.7230 -- iter: 0960/1000
Training Step: 80  | total loss: 0.75603 | time: 1.420s
| Adam | epoch: 005 | loss: 0.75603 - acc: 0.7273 -- iter: 1000/1000
--
Time taken -  11.697669267654419


In [13]:
mod.save('cnn.tfl')

INFO:tensorflow:C:\Users\Mihir Ahuja\Desktop\Code\cnn.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [20]:
vocab_size

375

In [ ]:
q